In [3]:
import sys
# import FrEIA framework
sys.path.append("/FrEIA-master") 

In [4]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import figure
from torch.utils.data import DataLoader
import plain_inn_swap
from tesladatainn import TeslaDatasetInn
import numpy as np
import time

### Constants

In [5]:
batch_size = 2048
lr = 1e-5
device = "cuda:0"

num_features = 100
num_blocks = 5

# Lagrange multipliers of our multi-objective function
#Todo: those have to be optimized via wandb
lambd_predict_back= 1. 
lambd_latent = 1. #300.
lambd_rev = 1. #450.

### Training
The following cell basically trains an Invertible Neural Network as of (Ardizzone, 2018) without padding: `[y,z] <=> x` with `x` = parameters, `y` = temperature readings and `z` our latent variable (iid. Gaussian)

In [6]:
ds = TeslaDatasetInn(device = device, data = "train")
train_dataloader = DataLoader(ds, batch_size=batch_size, shuffle=False)
# get one sample of our dataset to infer its input and output dimension
x,y = ds.__getitem__(0)
dim_inp = x.shape[0]
dim_outp = 1

In [7]:
my_inn = plain_inn_swap.INN(ndim_tot = dim_inp, ndim_y = dim_outp, ndim_x = dim_inp, ndim_z=dim_inp-dim_outp, 
            device = device,
             lambd_predict_back = lambd_predict_back, lambd_latent = lambd_latent, lambd_rev = lambd_rev,
             feature = num_features, num_blocks = num_blocks, batch_size = batch_size, lr = lr)

In [ ]:
begin = time.time()
my_inn.train(n_epochs=100, train_loader=train_dataloader, val_loader=train_dataloader)
end = time.time()
#print("training time:", end - begin)

### Analysis of the model on training data

In [ ]:
x,y = ds.x_norm,ds.y

# Make a prediction
pred,_ = my_inn.model(x)
pred = pred.detach().cpu().numpy()[:,0] # take temperature components, the other three elements are z

#pred = (pred + 1)*(ds.ub_y.cpu().numpy() - ds.lb_y.cpu().numpy())/2 + ds.lb_y.cpu().numpy()
y = y.detach().cpu().numpy().squeeze()

print('relative l2 norm: ', np.linalg.norm(y-pred) / np.linalg.norm(y)*100)

In [ ]:
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

figure(figsize=(10, 8), dpi= 350)

plt.plot(pred, '--')
plt.plot(ds.y.cpu(), '-')
plt.legend(['Prediction', 'ground-truth'])
plt.ylabel('Temperature / °C')
plt.show()

### Function for evaluating the performance of the model on test data

In [21]:
def evaluate(id):
    
    dss = TeslaDatasetInn(device = device, ID = id, data = "test")

    x,y = dss.x_norm,dss.y

    # Make a prediction
    pred,_ = my_inn.model(x.to(device))
    pred = pred.detach().cpu().numpy()[:,0] # take temperature components, the other three elements are z

    #pred = (pred + 1)*(dss.ub_y - dss.lb_y)/2 + dss.lb_y
    y = y.detach().cpu().numpy().squeeze()

    print('relative l2 norm: ', np.linalg.norm(y-pred) / np.linalg.norm(y)*100)

    import matplotlib as mpl
    mpl.rcParams['agg.path.chunksize'] = 10000
    figure(figsize=(10, 8), dpi= 1440)

    plt.plot(pred, '--')
    plt.plot(y, '-')
    plt.legend(['Prediction', 'ground-truth'])
    plt.ylabel('Temperature / °C')
    plt.show()

### Analysis of the model on test data

In [ ]:
# Test values = [16,39,47,52,72,81,88]
evaluate(16)

In [ ]:
# Test values = [16,39,47,52,72,81,88]
evaluate(39)

In [ ]:
# Test values = [16,39,47,52,72,81,88]
evaluate(47)

In [ ]:
# Test values = [16,39,47,52,72,81,88]
evaluate(52)

In [ ]:
# Test values = [16,39,47,52,72,81,88]
evaluate(72)

In [ ]:
# Test values = [16,39,47,52,72,81,88]
evaluate(81)

In [ ]:
# Test values = [16,39,47,52,72,81,88]
evaluate(88)